In [ ]:
SERVER_ACCOUNT_KEY_PATH = "config/service_account_cred.json"
SECRETS_OAUTH_2_FILE_PATH = "config/client_secret.json"

BUCKET_API = "martech-sandbox"
GCS_CONNECT_FILE_PATH = "katia/utils/gcs_connect_sandbox_katia.py"
CLIENT_CONFIG_DATA = "clients/001/input_file.xlsx" 
#TEMPLATE_PROPERTY_ID = '360045070'
ACCOUNT_ID = '41128675'

In [ ]:
# import python libs
from google.cloud import storage
from google.oauth2.service_account import Credentials
import pandas as pd

def create_gcs_connection():
    creds = Credentials.from_service_account_file(SERVER_ACCOUNT_KEY_PATH)
    client = storage.Client(credentials=creds)
    get_gcs_bucket = client.get_bucket(BUCKET_API)
    gcs_blob = get_gcs_bucket.blob(GCS_CONNECT_FILE_PATH)
    contents_gcs = gcs_blob.download_as_string()
    return contents_gcs

def read_gcs_bucket_files(connect_gcs):
    contents = [
        connect_gcs.read_backend_api_method(),
        connect_gcs.read_api_request(),
        connect_gcs.read_backend_api_utils(),
        connect_gcs.read_backend_api_settings(),
    ]
    return contents


if __name__ == "__main__":

    ####### READ BACKEND FILES #####
    exec(create_gcs_connection())
    print("✅ GCS connected ✅")
    
    connect_gcs = GoogleCloudeStorage(SERVER_ACCOUNT_KEY_PATH)
    backend_contents = read_gcs_bucket_files(connect_gcs)
    
    for content in backend_contents:
        exec(content)

    backend_client_config = connect_gcs.read_client_config(CLIENT_CONFIG_DATA)
    print("✅ Backend files read ✅")
    
    ####### READ CONFIG VARIABLES FOR API SETUP #####
    client_id = settings_api.get("client_id")
    client_secret = settings_api.get("clent_secret")
    api_key = settings_api.get("api_key")

    ####### SETUP GOOGLE ANALYTICS API #####
    df_ga4_propety_ids = pd.read_excel(backend_client_config, sheet_name="Property")
    property_id_list = df_ga4_propety_ids["property_id"].tolist()

    ga4_oper = GoogleAnalyticsOperator(
        client_id, client_secret, SECRETS_OAUTH_2_FILE_PATH, property_id_list, api_key
    )
    access_token = ga4_oper.create_access_token()
    

In [ ]:
#ga4_oper.set_api_action()
    
template_api_url = f'https://analyticsadmin.googleapis.com/v1alpha/accounts/{ACCOUNT_ID}/accessBindings?key={api_key}'

#template_api_url = f'https://analyticsadmin.googleapis.com/v1alpha/properties/360045070/accessBindings?key={api_key}'

users = read_api_obj(template_api_url, access_token)



In [ ]:
def create_df(output, api_action):
    """
    Write data to an Excel sheet.
    
    Args:
        output (list of dictionaries): The data to be written to the Excel sheet.
        api_action (str): The name of the sheet in the Excel file.
        excel_writer (pd.ExcelWriter): The Excel writer object.
    """
    
    result = json.loads(output)

    # Extract the data
    data = result[api_action]

    # Create a DataFrame from the data
    df = pd.DataFrame(data)
    
    return df

In [ ]:
prerty_list_df = create_df(users, "accessBindings")

In [ ]:
print(prerty_list_df)

In [ ]:
import pandas as pd

users_df = prerty_list_df[['user', 'roles']]

print(users_df)

users_df.to_excel("output.xlsx")